**1. Grey Wolf Optimization using relu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**2. Grey Wolf Optimization using sigmoid Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='sigmoid', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")


**3. Grey Wolf Optimization using Squared Sine Activation  function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Custom Squared Sine Activation Function
class SquaredSineActivation(tf.keras.layers.Layer):
    def __init__(self, omega=1.0, **kwargs):
        super(SquaredSineActivation, self).__init__(**kwargs)
        self.omega = omega

    def call(self, inputs):
        return tf.sin(self.omega * inputs) ** 2

# CNN architecture with Squared Sine Activation
def create_cnn_model(activation_function):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=activation_function, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    model.add(Dense(10, activation='softmax'))
    return model
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**4. Grey Wolf Optimization using tanh Activation function**


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='tanh', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**5. Grey Wolf Optimization using SoftSign  Activation function**


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='softsign', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='softsign'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**6. Grey Wolf Optimization using SoftPlus Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='softplus', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='softplus'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**7.Grey Wolf Optimization using swish Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='swish', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='swish'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**8. Grey Wolf Optimization using mish Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='mish', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='mish'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**9. Grey Wolf Optimization using HardSigmoid Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Define the Hard Sigmoid Activation Layer
class HardSigmoidActivation(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(HardSigmoidActivation, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.keras.backend.hard_sigmoid(inputs)

def create_cnn_model(activation_function):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=activation_function, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    model.add(Dense(10, activation='softmax'))
    return model
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**10. Grey Wolf Optimization using selu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='selu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='selu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**11. Grey Wolf Optimization using elu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='elus', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='elu'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**12. Grey Wolf Optimization using PRelu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='PReLU', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='PReLU'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**13. Grey Wolf Optimization using LeakyRelu Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='LeakyReLU', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='LeakyReLU'))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

**14. Grey Wolf Optimization using GELU Activation function**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

# Define GELU activation function
def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh((tf.sqrt(2 / tf.constant(np.pi)) * (x + 0.044715 * x ** 3))))
    return x * cdf

# Loading MNIST data set
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Converting labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# CNN architecture with GELU Activation
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation=gelu, input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=gelu))
model.add(Dense(10, activation='softmax'))
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Defining the objective function for Grey Wolf Optimizer
def objective_function(params):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params[0]),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        batch_size=int(params[1]), epochs=int(params[2]), verbose=0)

    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    return [train_loss, 1 - train_accuracy]
# Defining the search space for GWO
param_boundaries = [
    (1e-6, 1e-2), # Learning rate
    (32, 128),     # Batch size
    (5, 50)        # Epochs
]
# Number of wolves in the swarm
num_wolves = 50
# Number of iterations
num_iterations = 100
# Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, num_wolves, objective_function, param_boundaries):
        self.num_wolves = num_wolves
        self.objective_function = objective_function
        self.param_boundaries = param_boundaries

    def optimize(self, num_iterations):
        # Initialization
        wolves = [self._initialize_wolf() for _ in range(self.num_wolves)]

        for iteration in range(num_iterations):
            for i in range(self.num_wolves):
                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[0] - wolves[i][0]  # Equation (3.6)
                C = 2 * c[0]  # Equation (3.8)
                D = abs(C * wolves[i][0] - A)  # Equation (3.10)
                X1 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[1] - wolves[i][1]  # Equation (3.6)
                C = 2 * c[1]  # Equation (3.8)
                D = abs(C * wolves[i][1] - A)  # Equation (3.10)
                X2 = A - self._get_random() * D  # Equation (3.7)

                a, b, c = self._select_three_random_wolves(i)
                A = 2 * a[2] - wolves[i][2]  # Equation (3.6)
                C = 2 * c[2]  # Equation (3.8)
                D = abs(C * wolves[i][2] - A)  # Equation (3.10)
                X3 = A - self._get_random() * D  # Equation (3.7)

                # Ensuring the search space boundaries
                X1 = self._clip(X1, *self.param_boundaries[0])
                X2 = self._clip(X2, *self.param_boundaries[1])
                X3 = self._clip(X3, *self.param_boundaries[2])

                # Updating the position of the current wolf
                new_position = [X1, X2, X3]
                new_fitness = self.objective_function(new_position)

                # Comparing the fitness of the new position with the current position
                if new_fitness < wolves[i][-1]:
                    wolves[i] = new_position + new_fitness

            # Sorting the wolves based on fitness
            wolves.sort(key=lambda x: x[-1])

        # Returning the best solution found
        return wolves[0][:-1]

    def _initialize_wolf(self):
        position = [self._get_random_in_range(*boundaries) for boundaries in self.param_boundaries]
        fitness = self.objective_function(position)
        return position + fitness

    def _select_three_random_wolves(self, current_index):
        indices = [i for i in range(self.num_wolves) if i != current_index]
        selected_indices = self._get_random_sample(indices, size=3)
        selected_wolves = [wolves[i] for i in selected_indices]
        return selected_wolves

    @staticmethod
    def _get_random():
        return tf.random.uniform(shape=(), minval=0, maxval=1)

    @staticmethod
    def _get_random_in_range(min_val, max_val):
        return tf.random.uniform(shape=(), minval=min_val, maxval=max_val)

    @staticmethod
    def _get_random_sample(population, size):
        return tf.random.shuffle(population)[:size]

    @staticmethod
    def _clip(value, min_val, max_val):
        return tf.clip_by_value(value, min_val, max_val)
# Creating a Grey Wolf Optimizer object
gwo = GreyWolfOptimizer(num_wolves, objective_function, param_boundaries)
# Finding the optimal parameters
optimal_params = gwo.optimize(num_iterations)
# Displaying the best parameters found
print(f"Best parameters found by GWO: {optimal_params}")
# Compiling the model with the best parameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_params[0]),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model with the best parameters
history = model.fit(X_train, y_train, batch_size=int(optimal_params[1]), epochs=int(optimal_params[2]), verbose=1)
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")